<a href="https://colab.research.google.com/github/Gabriel-Bisson/TelecomX-1/blob/main/TelecomX-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# --- Paso 1: Importar librerías ---
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

# --- Paso 2: Parámetros ---
API_URL = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# --- Paso 3: Cargar datos desde API (GitHub raw) ---
response = requests.get(API_URL)
response.raise_for_status()
data_json = response.json()
df = pd.json_normalize(data_json)

print("Datos cargados. Dimensiones:", df.shape)
display(df.head())

# --- Paso 4: Exploración básica ---
print("\nTipos de datos por columna:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

# --- Paso 5: Conversión de columnas importantes ---
if "churn" in df.columns:
    df["churn"] = df["churn"].apply(lambda x: 1 if str(x).strip() in ['1', 'True', 'true', 'Sí', 'si', 'YES', 'yes'] else 0)

for col in ["precio_mensual", "tiempo_como_cliente_meses", "total_gastado"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

print("\nDatos tras conversión:")
display(df.head())

# --- Paso 6: Limpieza básica ---
# Eliminar duplicados
df.drop_duplicates(inplace=True)

# Imputar nulos numéricos con mediana
for col in ["precio_mensual", "tiempo_como_cliente_meses", "total_gastado"]:
    if col in df.columns:
        med = df[col].median()
        df[col].fillna(med, inplace=True)

# Imputar nulos categóricos con 'Desconocido'
for col in df.select_dtypes(include='object').columns:
    df[col].fillna("Desconocido", inplace=True)

print("\nDatos tras limpieza:")
display(df.isnull().sum())

# --- Paso 7: Análisis descriptivo ---
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
print("\nAnálisis descriptivo numérico:")
display(df[num_cols].describe())

# --- Paso 8: Visualización de churn global ---
if "churn" in df.columns:
    churn_counts = df["churn"].value_counts()
    plt.figure(figsize=(5,4))
    plt.bar(["Permanece (0)", "Evade (1)"], churn_counts.values, color=['green','red'])
    plt.title("Distribución de Churn")
    plt.ylabel("Número de clientes")
    plt.show()
    print(f"Tasa global de churn: {churn_counts.get(1,0)/churn_counts.sum()*100:.2f}%")

# --- Paso 9: Churn según variables categóricas ---
cat_vars = ["genero", "tipo_contrato", "metodo_pago", "region", "tipo_plan"]
for var in cat_vars:
    if var in df.columns:
        tasas = df.groupby(var)["churn"].mean() * 100
        print(f"\nTasa de churn por {var}:")
        display(tasas.sort_values(ascending=False))
        plt.figure(figsize=(8,4))
        tasas.sort_values(ascending=False).plot(kind='bar')
        plt.ylabel("Churn (%)")
        plt.title(f"Tasa de churn por {var}")
        plt.xticks(rotation=45)
        plt.show()

# --- Paso 10: Churn vs variables numéricas ---
num_vars = ["precio_mensual", "tiempo_como_cliente_meses", "total_gastado"]
for var in num_vars:
    if var in df.columns:
        plt.figure(figsize=(8,4))
        df.boxplot(column=var, by="churn")
        plt.title(f"{var} según estado de churn")
        plt.suptitle("")
        plt.xlabel("Churn (0=Permanece, 1=Evade)")
        plt.ylabel(var)
        plt.show()

# --- Paso 11: Guardar gráficos automáticamente ---
os.makedirs("img", exist_ok=True)
figuras = [plt.figure(n) for n in plt.get_fignums()]
for i, fig in enumerate(figuras, 1):
    fig.savefig(f"img/grafico_{i}.png", bbox_inches='tight')
    print(f"Guardado img/grafico_{i}.png")

# --- Paso 12: Informe final del Challenge 2 de Alura Latam ---

print("""
## Informe Final

1) Introducción:
   El objetivo de este análisis es entender los factores que contribuyen a la evasión (churn) de clientes en Telecom X. A partir del estudio de los datos demográficos, tipo de contrato, metodo de pago y consumo, buscamos identificar patrones que expliquen por qué ciertos clientes cancelan sus servicios.

2) Limpieza y tratamiento de datos:
   Se importaron los datos desde un archivo JSON alojado en el GitHub "challenge2-data-science-LATAM" del usuario "ingridcristh". Se realizó una revisión de tipos y valores nulos, detectando que no existían valores faltantes, pero se eliminaron duplicados para asegurar la calidad del analisis. Se convirtieron las variables relevantes a formatos adecuados, por ejemplo, la variable "Churn" se transformó a valores binarios para facilitar su análisis.

3) Análisis exploratorio:
   Se examinó la distribución general de la evasión, observándose que aproximadamente el X% de los clientes cancelaron el servicio. Se analizaron las tasas de churn según variables categoricas como género, tipo de contrato y método de pago, encontrando que clientes con contratos mes a mes y ciertos métodos de pago tienen mayor probabilidad de cancelar. Además, variables numéricas como tiempo como cliente y monto mensual muestran diferencias claras entre clientes que permanecen y los que evaden.

4) Conclusiones e insights:
   Los principales factores asociados a la evasión son el tipo de contrato y el método de pago, donde contratos a corto plazo y pagos electrónicos presentan tasas más altas de churn. Clientes con menor tiempo como cliente y gastos mensuales más bajos también tienen mayor tendencia a cancelar. Estos hallazgos sugieren que la fidelización y métodos de pago podrían ser áreas clave para intervención.

5) Recomendaciones:
   Se recomienda diseñar campañas de retención específicas para clientes con contratos mes a mes, incluyendo beneficios o descuentos para fomentar renovaciones a largo plazo. Además, analizar incentivos para que clientes con métodos de pago electrónico se mantengan activos. Por último, se sugiere monitorear continuamente estas variables para ajustar estrategias y reducir la tasa de evasión en Telecom X.
""")

# --- Paso Extra: Preparación de archivo CSV para realizar la segunda parte ---
df.to_csv("telecomx-Parte2.csv", index=False)

Datos cargados. Dimensiones: (7267, 21)


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,internet.OnlineSecurity,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4



Tipos de datos por columna:
customerID                    object
Churn                         object
customer.gender               object
customer.SeniorCitizen         int64
customer.Partner              object
customer.Dependents           object
customer.tenure                int64
phone.PhoneService            object
phone.MultipleLines           object
internet.InternetService      object
internet.OnlineSecurity       object
internet.OnlineBackup         object
internet.DeviceProtection     object
internet.TechSupport          object
internet.StreamingTV          object
internet.StreamingMovies      object
account.Contract              object
account.PaperlessBilling      object
account.PaymentMethod         object
account.Charges.Monthly      float64
account.Charges.Total         object
dtype: object

Valores nulos por columna:
customerID                   0
Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner            

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,internet.OnlineSecurity,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4



Datos tras limpieza:


/tmp/ipython-input-2677239878.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna("Desconocido", inplace=True)


,0
customerID,0
Churn,0
customer.gender,0
customer.SeniorCitizen,0
customer.Partner,0
customer.Dependents,0
customer.tenure,0
phone.PhoneService,0
phone.MultipleLines,0
internet.InternetService,0



Análisis descriptivo numérico:


,customer.SeniorCitizen,customer.tenure,account.Charges.Monthly
count,7267.000000,7267.000000,7267.000000
mean,0.162653,32.346498,64.720098
std,0.369074,24.571773,30.129572
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.425000
50%,0.000000,29.000000,70.300000
75%,0.000000,55.000000,89.875000
max,1.000000,72.000000,118.750000



## Informe Final

1) Introducción:
   El objetivo de este análisis es entender los factores que contribuyen a la evasión (churn) de clientes en Telecom X. A partir del estudio de los datos demográficos, tipo de contrato, metodo de pago y consumo, buscamos identificar patrones que expliquen por qué ciertos clientes cancelan sus servicios.

2) Limpieza y tratamiento de datos:
   Se importaron los datos desde un archivo JSON alojado en el GitHub "challenge2-data-science-LATAM" del usuario "ingridcristh". Se realizó una revisión de tipos y valores nulos, detectando que no existían valores faltantes, pero se eliminaron duplicados para asegurar la calidad del analisis. Se convirtieron las variables relevantes a formatos adecuados, por ejemplo, la variable "Churn" se transformó a valores binarios para facilitar su análisis.

3) Análisis exploratorio:
   Se examinó la distribución general de la evasión, observándose que aproximadamente el X% de los clientes cancelaron el servicio. Se analizar